In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [ ]:
with open("test_input.txt", "r") as f:
    s = f.read()

In [ ]:
with open("input.txt", "r") as f:
    s = f.read()

In [ ]:
def expand_coords(start_coord, end_coord):
    dx = end_coord[0] - start_coord[0]
    dy = end_coord[1] - start_coord[1]
    assert not all([dx==0, dy==0])
    if dx!=0 and dy==0:
        return [(x, start_coord[1]) for x in range(start_coord[0], end_coord[0], dx//abs(dx))]
    elif dx==0 and dy!=0:
        return [(start_coord[0], y) for y in range(start_coord[1], end_coord[1], dy//abs(dy))]
    elif dx==0 and dy==0:
        return start_coord

In [ ]:
rocks = defaultdict(dict)
rocks[500][0] = "+"

In [ ]:
for row in s.splitlines():
    line = []
    coords = [
        tuple(int(n) for n in coord.split(","))
        for coord in row.split(" -> ")
    ]
    for i, coord in enumerate(coords):
        if i+1 < len(coords):
            line.extend(expand_coords(coord, coords[i+1]))
        else:
            line.append(coord)
    for (x,y) in line:
        rocks[x][y] = "#"

In [ ]:
def fill_empty(res):
    return res.reindex(
        list(range(res.index.min(), res.index.max()+1)), 
        columns=list(range(res.columns.min(), res.columns.max()+1)), 
        fill_value="."
    )

In [ ]:
res = pd.DataFrame(rocks).sort_index(axis=0).sort_index(axis=1).fillna(".")
res = fill_empty(res)
res

In [ ]:
# For part two
height = res.index.max()
# y (horizontal) axis for the "infinite" plane. Physically it shouldn't need more than this
inf_plane_y = list(range(500-2*height, 500+2*height))
for y in inf_plane_y:
    res.loc[(height+2, y)] = "#"
res = fill_empty(res.sort_index(axis=0).sort_index(axis=1).fillna("."))
res

In [ ]:
def vec_add(vec_a: tuple, vec_b: tuple) -> tuple:
    return tuple(np.array(vec_a) + np.array(vec_b))

In [ ]:
class OutOfBoundError(Exception):
    def __init__(self):
        super().__init__("Drop out of bound")

In [ ]:
def try_move(res, vec):
    try:
        next = res.loc[vec]
        if next == ".":
            return True
        else:
            # Could be stone (o), wall (#), pouring point (+)
            return False
    except KeyError:
        # The stone dropes out of boundary
        raise OutOfBoundError

In [ ]:
def move_one_step(res, coord):
    # Note that the coord system used in .loc is different than the requirement
    vec_down = (1,0)
    vec_left_down = (1,-1)
    vec_right_down = (1,1)
    if try_move(res, vec_add(coord, vec_down)):
        # The stone could still move down
        move_one_step(res, vec_add(coord, vec_down))
    elif try_move(res, vec_add(coord, vec_left_down)):
        # The stone could still move to left down
        move_one_step(res, vec_add(coord, vec_left_down))
    elif try_move(res, vec_add(coord, vec_right_down)):
        # The stone could still move to right down
        move_one_step(res, vec_add(coord, vec_right_down))
    elif res.loc[coord] == "+":
        # Stones have piled up all the way to the top
        res.loc[coord] = "o"
        raise OutOfBoundError
    else:
        # The stone could not be moved at all, and none of the 3 positions are out of boundary
        res.loc[coord] = "o"

In [ ]:
try:
    while True:
        move_one_step(res, (0, 500))
except OutOfBoundError:
    print((res=="o").sum().sum())